# 제품 이상여부 판별 프로젝트

In [1171]:
import numpy as np
import pandas as pd

In [1172]:
RANDOM_STATE = 110

train_data = pd.read_csv("./final_data/train.csv")
test_data = pd.read_csv("./final_data/test.csv")

### 0. 결측값 처리

In [1173]:
# def shift_row_values(row, start_col_index, move_limit, total_columns):
#     move_count = 0  # 이동 카운터 초기화
#     for col_index in range(start_col_index, total_columns):  # 모든 열을 대상으로
#         if pd.isna(row[col_index]) or row[col_index] == "OK":  # 빈값 또는 "OK" 확인
#             # 빈값 또는 "OK"가 발견되면 현재 위치부터 이후 3칸 간격의 변수 값을 앞으로 이동
#             for shift_index in range(col_index, total_columns - 3, 3):  # 3칸씩 이동
#                 # 값을 이동
#                 row[shift_index] = row[shift_index + 3]
#                 row[shift_index + 3] = None  # 원래 자리 비우기
#                 move_count += 1  # 이동 카운트 증가

#                 if move_count >= move_limit:  # 설정된 횟수에 도달하면 중지
#                     break
#         if move_count >= move_limit:  # 외부 루프에서도 체크
#             break
#     return row

# def shift_values(data, start_col_index, move_limit):
#     total_columns = data.shape[1]
#     data = data.apply(shift_row_values, axis=1, args=(start_col_index, move_limit, total_columns))
#     return data

# # 변수 이름 설정 및 시작 열 인덱스 및 이동 횟수 설정
# variables_with_limits = [
#     ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam', 52),
#     ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1', 22),
#     ('HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2', 22)
# ]

# # 각 변수에 대해 함수 호출
# def process_data(data, variables_with_limits, output_file):
#     for start_var, move_limit in variables_with_limits:
#         start_col_index = data.columns.get_loc(start_var)  # 각 변수의 시작 열 인덱스 찾기
#         data = shift_values(data, start_col_index, move_limit)
#     data.to_csv(output_file, index=False)
#     print(f'데이터가 성공적으로 수정되고 저장되었습니다: {output_file}')

# # 데이터 처리
# process_data(train_data, variables_with_limits, './data/clean_train_data.csv')
# process_data(test_data, variables_with_limits, './data/clean_test_data.csv')

In [1174]:
RANDOM_STATE = 110

train_data = pd.read_csv("./final_data/clean_final_train.csv")
test_data = pd.read_csv("./final_data/clean_final_test.csv")
plus_data = pd.read_excel('./final_data/hand_data.xlsx')

### 1. 기본 전처리

In [1175]:
# target 열을 임시로 분리
target_train = train_data['target']
target_test = test_data['target']

# 모든 값이 NaN인 열 제거
train_data = train_data.dropna(axis=1, how='all')
test_data = test_data.dropna(axis=1, how='all')

# target 열을 다시 결합
train_data['target'] = target_train
test_data['target'] = target_test

In [1176]:
# Wip Line 열 제거
wip_line_columns = train_data.filter(like='Wip Line').columns

train_data.drop(columns=wip_line_columns, inplace=True)
test_data.drop(columns=wip_line_columns, inplace=True)

In [1177]:
# Process Desc 열 제거
Process_Desc_col = train_data.filter(like='Process Desc').columns

train_data.drop(columns=Process_Desc_col, inplace=True)
test_data.drop(columns=Process_Desc_col, inplace=True)

In [1178]:
# Insp. Seq No 열 제거
Insp_Seq_No_col = train_data.filter(like='Insp. Seq No').columns

train_data.drop(columns=Insp_Seq_No_col, inplace=True)
test_data.drop(columns=Insp_Seq_No_col, inplace=True)

In [1179]:
# Insp Judge Code 열 제거
Insp_Judge_Code_col = train_data.filter(like='Insp Judge Code').columns

train_data.drop(columns=Insp_Judge_Code_col, inplace=True)
test_data.drop(columns=Insp_Judge_Code_col, inplace=True)

### 2. 제품 구분

- receip no, workorder, model.suffix

Receip_No

In [1180]:
### Receip_No
# 파생변수 생성: Receip_No 3개의 컬럼 값이 모두 동일하면 해당 값을 저장, 아니면 diff
train_data['Receip_No'] = train_data.apply(
    lambda row: row['Receip No Collect Result_Dam'] if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) else 'diff',
    axis=1
)
test_data['Receip_No'] = test_data.apply(
    lambda row: row['Receip No Collect Result_Dam'] if (row['Receip No Collect Result_Dam'] == row['Receip No Collect Result_Fill1'] == row['Receip No Collect Result_Fill2']) else 'diff',
    axis=1
)

# receip_no 열을 object 타입으로 변경
train_data['Receip_No'] = train_data['Receip_No'].astype('object')
test_data['Receip_No'] = test_data['Receip_No'].astype('object')

# 필요없는 변수 삭제
train_data = train_data.drop(columns=['Receip No Collect Result_Dam', 'Receip No Collect Result_Fill1', 'Receip No Collect Result_Fill2'])
test_data = test_data.drop(columns=['Receip No Collect Result_Dam', 'Receip No Collect Result_Fill1', 'Receip No Collect Result_Fill2'])

workorder_receip

In [1181]:
### workorder_receip
# Workorder -뒤의 번호 구분을 제거
train_data['cleaned_workorder'] = train_data['Workorder_Dam'].str.split('-').str[0]
test_data['cleaned_workorder'] = test_data['Workorder_Dam'].str.split('-').str[0]

# 필요없는 변수 삭제
train_data = train_data.drop(columns=['Workorder_Dam', 'Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'])
test_data = test_data.drop(columns=['Workorder_Dam', 'Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'])

model_suffix

In [1182]:
### model_suffix
# 열 이름 변경
train_data.rename(columns={'Model.Suffix_Dam': 'model_suffix'}, inplace=True)
test_data.rename(columns={'Model.Suffix_Dam': 'model_suffix'}, inplace=True)

# 필요없는 변수 삭제
train_data = train_data.drop(columns=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2'])
test_data = test_data.drop(columns=['Model.Suffix_AutoClave', 'Model.Suffix_Fill1', 'Model.Suffix_Fill2'])

In [1183]:
# model_suffix 값이 'AJX75334503'인 경우 1, 아닌 경우 0을 부여하는 파생변수 생성
train_data['model_suffix_03'] = train_data['model_suffix'].apply(lambda x: 1 if x == 'AJX75334503' else 0)
test_data['model_suffix_03'] = test_data['model_suffix'].apply(lambda x: 1 if x == 'AJX75334503' else 0)

In [1184]:
# Receip_No가 3이고 model_suffix_03이 1인(model_suffix가 3번) 경우 
# 1을 부여하고 아닌 경우 0을 부여하는 파생변수 생성
train_data['Receip_n_suffix_3'] = train_data.apply(lambda row: 1 if row['Receip_No'] == 3 and row['model_suffix_03'] == 1 else 0, axis=1)
test_data['Receip_n_suffix_3'] = test_data.apply(lambda row: 1 if row['Receip_No'] == 3 and row['model_suffix_03'] == 1 else 0, axis=1)

model_suffix_03, Receip_No 변수 제거

In [1185]:
train_data = train_data.drop(columns=['model_suffix_03', 'Receip_No'])
test_data = test_data.drop(columns=['model_suffix_03', 'Receip_No'])

### 3. 공통 변수 (dam, fill1, fill2)

- workmode

In [1186]:
# WorkMode Collect Result_Dam의 이름을 WorkMode Collect Result로 변경
train_data = train_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})
test_data = test_data.rename(columns={'WorkMode Collect Result_Dam': 'WorkMode Collect Result'})

# WorkMode Collect Result_Fill1, WorkMode Collect Result_Fill2 열 드롭
train_data = train_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])
test_data = test_data.drop(columns=['WorkMode Collect Result_Fill1', 'WorkMode Collect Result_Fill2'])

In [1187]:
# WorkMode Collect Result 열의 값이 7인 행을 1로 변경
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].replace(7, 1)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].replace(7, 1)

# WorkMode Collect Result 열의 결측값을 0으로 채움
train_data['WorkMode Collect Result'] = train_data['WorkMode Collect Result'].fillna(0)
test_data['WorkMode Collect Result'] = test_data['WorkMode Collect Result'].fillna(0)

- equipment
<br>(dispenser1 & dispenser2 변수를 만들 경우 다른 변수들에 의해 이미 설명이 되는 변수라 상관계수가 너무 높아서 제거하게 됨. 따라서 equipment가 같은지만 판단하는 파생변수 사용)

In [1188]:
# Equipment로 시작하는 열 필터링
Equipment_col = train_data.filter(like='Equipment').columns
Equipment_col2 = test_data.filter(like='Equipment').columns

new_train = train_data.filter(items=Equipment_col)
new_test = test_data.filter(items=Equipment_col2)

# Equipment_same_num 파생변수 생성
def determine_equipment_same_num(row):
    if (row['Equipment_Dam'] == 'Dam dispenser #1' and row['Equipment_AutoClave'] == 'Auto Clave Out' and 
        row['Equipment_Fill1'] == 'Fill1 dispenser #1' and row['Equipment_Fill2'] == 'Fill2 dispenser #1') or \
       (row['Equipment_Dam'] == 'Dam dispenser #2' and row['Equipment_AutoClave'] == 'Auto Clave Out' and 
        row['Equipment_Fill1'] == 'Fill1 dispenser #2' and row['Equipment_Fill2'] == 'Fill2 dispenser #2'):
        return 1
    else:
        return 0

train_data['Equipment_same_num'] = new_train.apply(determine_equipment_same_num, axis=1)
test_data['Equipment_same_num'] = new_test.apply(determine_equipment_same_num, axis=1)

train_data = train_data.drop(columns=['Equipment_Dam', 'Equipment_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2'])
test_data = test_data.drop(columns=['Equipment_Dam', 'Equipment_AutoClave', 'Equipment_Fill1', 'Equipment_Fill2'])

- palletID

In [1189]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 diff의 값을 가지는 파생 변수 생성 함수
def create_palletid_collect_result(df):
    df['PalletID_Collect_Result'] = df.apply(
        lambda row: row['PalletID Collect Result_Dam'] 
                    if (row['PalletID Collect Result_Dam'] == row['PalletID Collect Result_Fill1'] == row['PalletID Collect Result_Fill2']) 
                    else 'diff', 
        axis=1
    )

# 함수 적용
create_palletid_collect_result(train_data)
create_palletid_collect_result(test_data)

In [1190]:
# 제거할 변수 목록
columns_to_drop = [
    'PalletID Collect Result_Dam',
    'PalletID Collect Result_Fill1',
    'PalletID Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

- production Qty

In [1191]:
# 세 변수의 값이 동일하면 해당 값을 가져가고, 하나라도 일치하지 않으면 0의 값을 가지는 파생 변수 생성 함수
def create_palletid_collect_result(df):
    df['Production_Qty_Collect_Result'] = df.apply(
        lambda row: row['Production Qty Collect Result_Dam'] 
                    if (row['Production Qty Collect Result_Dam'] == row['Production Qty Collect Result_Fill1'] == row['Production Qty Collect Result_Fill2']) 
                    else 'diff', 
        axis=1
    )

# 함수 적용
create_palletid_collect_result(train_data)
create_palletid_collect_result(test_data)

In [1192]:
# 제거할 변수 목록
columns_to_drop = [
    'Production Qty Collect Result_Dam',
    'Production Qty Collect Result_Fill1',
    'Production Qty Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 4. CURE 변수
- dam -> distance 파생변수 (standby는 단일값, start와 end는 값은 여러개지만 distance 파생변수를 만들었을 때 더 의미있었음)
- fill2 -> 변수값 범주화 (start, end, standby를 각각 범주화했을 때가 합쳐서 distance 만들었을 때보다 더 의미있었음)

In [1193]:
### dam
# 시작 위치와 끝 위치 열 이름
start_x_col = 'CURE START POSITION X Collect Result_Dam'
start_z_col = 33.5
end_x_col = 'CURE END POSITION X Collect Result_Dam'
end_z_col = 'CURE END POSITION Z Collect Result_Dam'

# 시작 위치와 끝 위치 사이의 거리 계산
train_data['CURE_DISTANCE_Dam'] = np.sqrt(
    (train_data[end_x_col] - train_data[start_x_col]) ** 2 +
    (train_data[end_z_col] - start_z_col) ** 2
)

test_data['CURE_DISTANCE_Dam'] = np.sqrt(
    (train_data[end_x_col] - train_data[start_x_col]) ** 2 +
    (train_data[end_z_col] - start_z_col) ** 2
)

In [1194]:
### fill2
# UV 경화 좌표 합치기
def create_coordinate_columns(data):
    # Fill2
    # cure end
    data['cure_end_position_XZ_Fill2'] = (
        data['CURE END POSITION X Collect Result_Fill2'].astype(str) + ',' +
        data['CURE END POSITION Z Collect Result_Fill2'].astype(str) 
    )

    # cure start
    data['cure_start_position_XZ_Fill2'] = (
        data['CURE START POSITION X Collect Result_Fill2'].astype(str) + ',' +
        data['CURE START POSITION Z Collect Result_Fill2'].astype(str) 
    )

    # cure standby
    data['cure_standby_position_XZ_Fill2'] = (
        data['CURE STANDBY POSITION X Collect Result_Fill2'].astype(str) + ',' +
        data['CURE STANDBY POSITION Z Collect Result_Fill2'].astype(str) 
    )

# train_data와 test_data에 대해 함수 호출
create_coordinate_columns(train_data)
create_coordinate_columns(test_data)

In [1195]:
# 제거할 변수 목록
columns_to_drop = [
    'CURE END POSITION X Collect Result_Dam',
    'CURE END POSITION Z Collect Result_Dam',
    'CURE END POSITION Θ Collect Result_Dam',
    'CURE START POSITION X Collect Result_Dam',
    'CURE START POSITION Z Collect Result_Dam',
    'CURE START POSITION Θ Collect Result_Dam',

    'CURE END POSITION X Collect Result_Fill2',
    'CURE END POSITION Z Collect Result_Fill2',
    'CURE END POSITION Θ Collect Result_Fill2',
    'CURE START POSITION X Collect Result_Fill2',
    'CURE START POSITION Z Collect Result_Fill2',
    'CURE START POSITION Θ Collect Result_Fill2',
    'CURE STANDBY POSITION X Collect Result_Fill2',
    'CURE STANDBY POSITION Z Collect Result_Fill2',
    'CURE STANDBY POSITION Θ Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 5. HEAD 변수

- dam

In [1196]:
# 각 스테이지의 좌표 열 정의
stage1_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam']

stage2_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam']

stage3_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam']

# 거리 계산 함수
def calculate_distances(data):
    data['HEAD NORMAL DISTANCE_STAGE1_STAGE2_Dam'] = np.sqrt(
        (data[stage2_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage2_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage2_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    data['HEAD NORMAL DISTANCE_STAGE2_STAGE3_Dam'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage2_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage2_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage2_cols[2]]) ** 2
    )

    data['HEAD NORMAL DISTANCE_STAGE1_STAGE3_Dam'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    return data

# train_data에 적용
train_data = calculate_distances(train_data)

# test_data에 적용
test_data = calculate_distances(test_data)

In [1197]:
# 필요한 열 이름
stage1_stage2_col = 'HEAD NORMAL DISTANCE_STAGE1_STAGE2_Dam'
stage2_stage3_col = 'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Dam'
stage1_stage3_col = 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Dam'

# 삼각형의 넓이와 높이를 계산하는 함수
def calculate_triangle_features(data):
    a = data[stage1_stage2_col]
    b = data[stage2_stage3_col]
    c = data[stage1_stage3_col]

    # 헤론의 공식에 따른 삼각형의 넓이 계산
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))

    # 높이 계산 (밑변을 c로 가정)
    height = (2 * area) / c

    # 결과를 새로운 열에 저장
    data['HEAD NORMAL DISTANCE_TRIANGLE_area_Dam'] = area
    data['HEAD NORMAL DISTANCE_TRIANGLE_height_Dam'] = height

    return data

# train_data에 적용
train_data = calculate_triangle_features(train_data)

# test_data에 적용
test_data = calculate_triangle_features(test_data)

In [1198]:
# 제거할 변수 목록
columns_to_drop = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'

    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'

    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'

    , 'HEAD NORMAL DISTANCE_STAGE1_STAGE2_Dam'
    , 'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Dam'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

In [1199]:
# Dam 노즐 zero 위치 Z좌표 드롭
train_data.drop(columns='Head Zero Position Z Collect Result_Dam', inplace=True)
test_data.drop(columns='Head Zero Position Z Collect Result_Dam', inplace=True)

- fill1

In [1200]:
# 각 스테이지의 좌표 열 정의
stage1_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1']

stage2_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1']

stage3_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1']

# 거리 계산 함수
def calculate_distances(data):
    data['HEAD NORMAL DISTANCE_STAGE1_STAGE2_Fill1'] = np.sqrt(
        (data[stage2_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage2_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage2_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    data['HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill1'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage2_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage2_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage2_cols[2]]) ** 2
    )

    data['HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill1'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    return data

# train_data에 적용
train_data = calculate_distances(train_data)

# test_data에 적용
test_data = calculate_distances(test_data)

In [1201]:
# 필요한 열 이름
stage1_stage2_col = 'HEAD NORMAL DISTANCE_STAGE1_STAGE2_Fill1'
stage2_stage3_col = 'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill1'
stage1_stage3_col = 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill1'

# 삼각형의 넓이와 높이를 계산하는 함수
def calculate_triangle_features(data):
    a = data[stage1_stage2_col]
    b = data[stage2_stage3_col]
    c = data[stage1_stage3_col]

    # 헤론의 공식에 따른 삼각형의 넓이 계산
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))

    # 높이 계산 (밑변을 c로 가정)
    height = (2 * area) / c

    # 결과를 새로운 열에 저장
    data['HEAD NORMAL DISTANCE_TRIANGLE_area_Fill1'] = area
    data['HEAD NORMAL DISTANCE_TRIANGLE_height_Fill1'] = height

    return data

# train_data에 적용
train_data = calculate_triangle_features(train_data)

# test_data에 적용
test_data = calculate_triangle_features(test_data)

In [1202]:
# 제거할 변수 목록
columns_to_drop = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'

    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'

    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'

    , 'HEAD NORMAL DISTANCE_STAGE1_STAGE2_Fill1'
    , 'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill1'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

- fill2

In [1203]:
# 각 스테이지의 좌표 열 정의
stage1_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2']

stage2_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2']

stage3_cols = ['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
               'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2']

# 거리 계산 함수
def calculate_distances(data):
    data['HEAD NORMAL DISTANCE_STAGE1_STAGE2_Fill2'] = np.sqrt(
        (data[stage2_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage2_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage2_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    data['HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill2'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage2_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage2_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage2_cols[2]]) ** 2
    )

    data['HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill2'] = np.sqrt(
        (data[stage3_cols[0]] - data[stage1_cols[0]]) ** 2 +
        (data[stage3_cols[1]] - data[stage1_cols[1]]) ** 2 +
        (data[stage3_cols[2]] - data[stage1_cols[2]]) ** 2
    )

    return data

# train_data에 적용
train_data = calculate_distances(train_data)

# test_data에 적용
test_data = calculate_distances(test_data)

In [1204]:
# 제거할 변수 목록
columns_to_drop = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'

    , 'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'

    , 'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'
    , 'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 6. Resin 변수

- dam

In [1205]:
# volume*time 파생변수 - Dam
train_data['volume_time_multip_stage1_Dam'] = train_data['Dispense Volume(Stage1) Collect Result_Dam'] * train_data['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
train_data['volume_time_multip_stage2_Dam'] = train_data['Dispense Volume(Stage2) Collect Result_Dam'] * train_data['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
train_data['volume_time_multip_stage3_Dam'] = train_data['Dispense Volume(Stage3) Collect Result_Dam'] * train_data['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

train_data['volume_time_multip_avg_Dam'] = (train_data['volume_time_multip_stage1_Dam'] + 
                                            train_data['volume_time_multip_stage2_Dam'] + 
                                            train_data['volume_time_multip_stage3_Dam']) / 3

# volume*time 파생변수 - Dam
test_data['volume_time_multip_stage1_Dam'] = test_data['Dispense Volume(Stage1) Collect Result_Dam'] * test_data['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
test_data['volume_time_multip_stage2_Dam'] = test_data['Dispense Volume(Stage2) Collect Result_Dam'] * test_data['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
test_data['volume_time_multip_stage3_Dam'] = test_data['Dispense Volume(Stage3) Collect Result_Dam'] * test_data['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

test_data['volume_time_multip_avg_Dam'] = (test_data['volume_time_multip_stage1_Dam'] + 
                                            test_data['volume_time_multip_stage2_Dam'] + 
                                            test_data['volume_time_multip_stage3_Dam']) / 3

In [1206]:
# 삭제할 열 목록 추가
columns_to_drop = [
    'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
    'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
    'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
    'Dispense Volume(Stage1) Collect Result_Dam',
    'Dispense Volume(Stage2) Collect Result_Dam',
    'Dispense Volume(Stage3) Collect Result_Dam',
    #'volume_time_multip_stage1_Dam',
    #'volume_time_multip_stage2_Dam',
    #'volume_time_multip_stage3_Dam'
]

train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

- fill1

In [1207]:
# volume*time 파생변수 - Fill1
train_data['volume_time_multip_stage1_Fill1'] = train_data['Dispense Volume(Stage1) Collect Result_Fill1'] * train_data['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
train_data['volume_time_multip_stage2_Fill1'] = train_data['Dispense Volume(Stage2) Collect Result_Fill1'] * train_data['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
train_data['volume_time_multip_stage3_Fill1'] = train_data['Dispense Volume(Stage3) Collect Result_Fill1'] * train_data['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

train_data['volume_time_multip_avg_Fill1'] = (train_data['volume_time_multip_stage1_Fill1'] + 
                                            train_data['volume_time_multip_stage2_Fill1'] + 
                                            train_data['volume_time_multip_stage3_Fill1']) / 3

# volume*time 파생변수 - Fill1
test_data['volume_time_multip_stage1_Fill1'] = test_data['Dispense Volume(Stage1) Collect Result_Fill1'] * test_data['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
test_data['volume_time_multip_stage2_Fill1'] = test_data['Dispense Volume(Stage2) Collect Result_Fill1'] * test_data['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
test_data['volume_time_multip_stage3_Fill1'] = test_data['Dispense Volume(Stage3) Collect Result_Fill1'] * test_data['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

test_data['volume_time_multip_avg_Fill1'] = (test_data['volume_time_multip_stage1_Fill1'] + 
                                            test_data['volume_time_multip_stage2_Fill1'] + 
                                            test_data['volume_time_multip_stage3_Fill1']) / 3

In [1208]:
# 삭제할 열 목록 추가
columns_to_drop = [
    'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
    'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
    'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
    'Dispense Volume(Stage1) Collect Result_Fill1',
    'Dispense Volume(Stage2) Collect Result_Fill1',
    'Dispense Volume(Stage3) Collect Result_Fill1',
    #'volume_time_multip_stage1_Fill1',
    #'volume_time_multip_stage2_Fill1',
    #'volume_time_multip_stage3_Fill1'
]

train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

### 7. Circle, Line 변수

In [1209]:
### circle
# 열 이름 변경
train_data.rename(columns={
    'Stage1 Circle1 Distance Speed Collect Result_Dam': 'Stage1_Circle_Distance_Speed_Dam',
    'Stage2 Circle1 Distance Speed Collect Result_Dam': 'Stage2_Circle_Distance_Speed_Dam',
    'Stage3 Circle1 Distance Speed Collect Result_Dam': 'Stage3_Circle_Distance_Speed_Dam'
}, inplace=True)

test_data.rename(columns={
    'Stage1 Circle1 Distance Speed Collect Result_Dam': 'Stage1_Circle_Distance_Speed_Dam',
    'Stage2 Circle1 Distance Speed Collect Result_Dam': 'Stage2_Circle_Distance_Speed_Dam',
    'Stage3 Circle1 Distance Speed Collect Result_Dam': 'Stage3_Circle_Distance_Speed_Dam'
}, inplace=True)

In [1210]:
# 제거할 변수 목록
columns_to_drop = [
    'Stage1 Circle2 Distance Speed Collect Result_Dam',
    'Stage1 Circle3 Distance Speed Collect Result_Dam',
    'Stage1 Circle4 Distance Speed Collect Result_Dam',
    
    'Stage2 Circle2 Distance Speed Collect Result_Dam',
    'Stage2 Circle3 Distance Speed Collect Result_Dam',
    'Stage2 Circle4 Distance Speed Collect Result_Dam',
    
    'Stage3 Circle2 Distance Speed Collect Result_Dam',
    'Stage3 Circle3 Distance Speed Collect Result_Dam',
    'Stage3 Circle4 Distance Speed Collect Result_Dam'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

In [1211]:
### line
# line1&3과 line2&4를 합친 파생변수 생성 함수
def check_distance_speed(data, stage):
    # 단계에 따라 라인 번호 정의
    line_pairs = [(1, 3), (2, 4)]
    
    # 각 라인 쌍에 대해 반복
    for line1, line2 in line_pairs:
        line1_name = f'Stage{stage} Line{line1} Distance Speed Collect Result_Dam'
        line2_name = f'Stage{stage} Line{line2} Distance Speed Collect Result_Dam'
        
        # 새로운 열 이름 설정
        new_col_name = f'stage{stage}_line{line1}{line2}_distance_speed_Dam'
        
        # 조건에 따라 값 설정
        data[new_col_name] = data.apply(
            lambda row: row[line1_name] if row[line1_name] == row[line2_name] else 'diff', axis=1
        )

# train_data와 test_data 모두에 대해 함수 호출
for stage in range(1, 4):
    check_distance_speed(train_data, stage)
    check_distance_speed(test_data, stage)

In [1212]:
# train_data에서 변수들을 object 타입으로 변환
train_data['stage1_line24_distance_speed_Dam'] = train_data['stage1_line24_distance_speed_Dam'].astype(object)
train_data['stage2_line24_distance_speed_Dam'] = train_data['stage2_line24_distance_speed_Dam'].astype(object)
train_data['stage3_line24_distance_speed_Dam'] = train_data['stage3_line24_distance_speed_Dam'].astype(object)

# test_data에서 변수들을 object 타입으로 변환
test_data['stage1_line24_distance_speed_Dam'] = test_data['stage1_line24_distance_speed_Dam'].astype(object)
test_data['stage2_line24_distance_speed_Dam'] = test_data['stage2_line24_distance_speed_Dam'].astype(object)
test_data['stage3_line24_distance_speed_Dam'] = test_data['stage3_line24_distance_speed_Dam'].astype(object)

In [1213]:
# 제거할 변수 목록
columns_to_drop = [
    'Stage1 Line1 Distance Speed Collect Result_Dam',
    'Stage1 Line2 Distance Speed Collect Result_Dam',
    'Stage1 Line3 Distance Speed Collect Result_Dam',
    'Stage1 Line4 Distance Speed Collect Result_Dam',
    
    'Stage2 Line1 Distance Speed Collect Result_Dam',
    'Stage2 Line2 Distance Speed Collect Result_Dam',
    'Stage2 Line3 Distance Speed Collect Result_Dam',
    'Stage2 Line4 Distance Speed Collect Result_Dam',
    
    'Stage3 Line1 Distance Speed Collect Result_Dam',
    'Stage3 Line2 Distance Speed Collect Result_Dam',
    'Stage3 Line3 Distance Speed Collect Result_Dam',
    'Stage3 Line4 Distance Speed Collect Result_Dam'
]

# 변수 제거
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 8. Thickness 변수

In [1214]:
# 세 개 컬럼의 평균을 계산하여 새로운 컬럼 생성
train_data['average_thickness_Dam'] = train_data[['THICKNESS 1 Collect Result_Dam', 
                                                  'THICKNESS 2 Collect Result_Dam', 
                                                  'THICKNESS 3 Collect Result_Dam']].mean(axis=1)

test_data['average_thickness_Dam'] = test_data[['THICKNESS 1 Collect Result_Dam', 
                                                'THICKNESS 2 Collect Result_Dam', 
                                                'THICKNESS 3 Collect Result_Dam']].mean(axis=1)

In [1215]:
# 삭제할 컬럼 리스트
columns_to_drop = [
    'THICKNESS 1 Collect Result_Dam',
    'THICKNESS 2 Collect Result_Dam',
    'THICKNESS 3 Collect Result_Dam'
]

# 지정한 컬럼 삭제
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 9. Autoclave 관련 변수

In [1216]:
# 각 압력과 시간의 곱을 담은 새로운 컬럼 생성
train_data['1st_pressure_time_AutoClave'] = train_data['1st Pressure Collect Result_AutoClave'] * train_data['1st Pressure 1st Pressure Unit Time_AutoClave']
train_data['2nd_pressure_time_AutoClave'] = train_data['2nd Pressure Collect Result_AutoClave'] * train_data['2nd Pressure Unit Time_AutoClave']
train_data['3rd_pressure_time_AutoClave'] = train_data['3rd Pressure Collect Result_AutoClave'] * train_data['3rd Pressure Unit Time_AutoClave']

train_data['avg_pressure_time_AutoClave'] = (train_data['1st_pressure_time_AutoClave'] +
                                             train_data['2nd_pressure_time_AutoClave'] +
                                             train_data['3rd_pressure_time_AutoClave']) / 3

In [1217]:
# 각 압력과 시간의 곱을 담은 새로운 컬럼 생성
test_data['1st_pressure_time_AutoClave'] = test_data['1st Pressure Collect Result_AutoClave'] * test_data['1st Pressure 1st Pressure Unit Time_AutoClave']
test_data['2nd_pressure_time_AutoClave'] = test_data['2nd Pressure Collect Result_AutoClave'] * test_data['2nd Pressure Unit Time_AutoClave']
test_data['3rd_pressure_time_AutoClave'] = test_data['3rd Pressure Collect Result_AutoClave'] * test_data['3rd Pressure Unit Time_AutoClave']

test_data['avg_pressure_time_AutoClave'] = (test_data['1st_pressure_time_AutoClave'] +
                                             test_data['2nd_pressure_time_AutoClave'] +
                                             test_data['3rd_pressure_time_AutoClave']) / 3

In [1218]:
# 삭제할 컬럼 리스트
columns_to_drop = [
    '1st Pressure Collect Result_AutoClave',
    '1st Pressure 1st Pressure Unit Time_AutoClave',
    '2nd Pressure Collect Result_AutoClave',
    '2nd Pressure Unit Time_AutoClave',
    '3rd Pressure Collect Result_AutoClave',
    '3rd Pressure Unit Time_AutoClave',
]

# 지정한 컬럼 삭제
train_data.drop(columns=columns_to_drop, inplace=True)
test_data.drop(columns=columns_to_drop, inplace=True)

### 10. Time 변수

In [1219]:
# 총시간 대비 비율 변수
def calculate_total_time_and_ratios(data):
    data['total_time'] = (
        data['Machine Tact time Collect Result_Dam'] +
        data['Machine Tact time Collect Result_Fill1'] +
        data['Machine Tact time Collect Result_Fill2'] +
        data['Chamber Temp. Unit Time_AutoClave']
    )
    data['time_ratio_Dam'] = (data['Machine Tact time Collect Result_Dam'] / data['total_time']).round(3)
    data['time_ratio_Fill1'] = (data['Machine Tact time Collect Result_Fill1'] / data['total_time']).round(3)
    data['time_ratio_Fill2'] = (data['Machine Tact time Collect Result_Fill2'] / data['total_time']).round(3)
    data['time_ratio_AutoClave'] = (data['Chamber Temp. Unit Time_AutoClave'] / data['total_time']).round(3)
    return data

# train_data와 test_data에 함수 적용
train_data = calculate_total_time_and_ratios(train_data)
test_data = calculate_total_time_and_ratios(test_data)

In [1220]:
# 변수 제거
train_data.drop(columns=[
    'total_time'
    , 'Machine Tact time Collect Result_Dam'
    , 'Machine Tact time Collect Result_Fill1'
    , 'Machine Tact time Collect Result_Fill2'
    , 'Chamber Temp. Unit Time_AutoClave'], inplace=True)

test_data.drop(columns=[
    'total_time'
    , 'Machine Tact time Collect Result_Dam'
    , 'Machine Tact time Collect Result_Fill1'
    , 'Machine Tact time Collect Result_Fill2'
    , 'Chamber Temp. Unit Time_AutoClave'], inplace=True)

### 00. 추가된 변수

In [1221]:
# 'Process Desc'를 포함하는 열 이름 필터링
filter_col = train_data.filter(like='Collect Date_').columns

print("< Process Desc 포함 변수 >")
for col in filter_col:
    print(col)

< Process Desc 포함 변수 >
Collect Date_Dam
Collect Date_AutoClave
Collect Date_Fill1
Collect Date_Fill2


In [1222]:
import pandas as pd

# Collect Date 열들을 datetime 형식으로 변환
train_data['Collect Date_Dam'] = pd.to_datetime(train_data['Collect Date_Dam'])
train_data['Collect Date_Fill1'] = pd.to_datetime(train_data['Collect Date_Fill1'])
train_data['Collect Date_Fill2'] = pd.to_datetime(train_data['Collect Date_Fill2'])
train_data['Collect Date_AutoClave'] = pd.to_datetime(train_data['Collect Date_AutoClave'])

test_data['Collect Date_Dam'] = pd.to_datetime(test_data['Collect Date_Dam'])
test_data['Collect Date_Fill1'] = pd.to_datetime(test_data['Collect Date_Fill1'])
test_data['Collect Date_Fill2'] = pd.to_datetime(test_data['Collect Date_Fill2'])
test_data['Collect Date_AutoClave'] = pd.to_datetime(test_data['Collect Date_AutoClave'])

# 각 열 간의 시간 차이를 계산하여 새로운 변수 생성
train_data['time_diff_Dam_Fill1'] = (train_data['Collect Date_Fill1'] - train_data['Collect Date_Dam']).dt.total_seconds() / 60
train_data['time_diff_Fill1_Fill2'] = (train_data['Collect Date_Fill2'] - train_data['Collect Date_Fill1']).dt.total_seconds() / 60
train_data['time_diff_Fill2_AutoClave'] = (train_data['Collect Date_AutoClave'] - train_data['Collect Date_Fill2']).dt.total_seconds() / 60
train_data['time_diff_Dam_AutoClave'] = (train_data['Collect Date_AutoClave'] - train_data['Collect Date_Dam']).dt.total_seconds() / 60 

test_data['time_diff_Dam_Fill1'] = (test_data['Collect Date_Fill1'] - test_data['Collect Date_Dam']).dt.total_seconds() / 60
test_data['time_diff_Fill1_Fill2'] = (test_data['Collect Date_Fill2'] - test_data['Collect Date_Fill1']).dt.total_seconds() / 60
test_data['time_diff_Fill2_AutoClave'] = (test_data['Collect Date_AutoClave'] - test_data['Collect Date_Fill2']).dt.total_seconds() / 60
test_data['time_diff_Dam_AutoClave'] = (test_data['Collect Date_AutoClave'] - test_data['Collect Date_Dam']).dt.total_seconds() / 60

In [1223]:
import pandas as pd

# time_diff_Dam_Fill1의 값이 20 이상이면 1, 아니면 0을 부여하는 파생 변수 생성
train_data['time_gap_Dam'] = train_data['time_diff_Dam_Fill1'].apply(lambda x: 1 if x >= 20 else 0)
test_data['time_gap_Dam'] = test_data['time_diff_Dam_Fill1'].apply(lambda x: 1 if x >= 20 else 0)

# time_diff_Fill1_Fill2의 값이 20 이상이면 1, 아니면 0을 부여하는 파생 변수 생성
train_data['time_gap_Fill1'] = train_data['time_diff_Fill1_Fill2'].apply(lambda x: 1 if x >= 20 else 0)
test_data['time_gap_Fill1'] = test_data['time_diff_Fill1_Fill2'].apply(lambda x: 1 if x >= 20 else 0)

# time_diff_Fill2_AutoClave의 값이 120 이상이면 1을 부여하는 파생 변수 생성
train_data['time_gap_Fill2'] = train_data['time_diff_Fill2_AutoClave'].apply(lambda x: 1 if x >= 120 else 0)
test_data['time_gap_Fill2'] = test_data['time_diff_Fill2_AutoClave'].apply(lambda x: 1 if x >= 120 else 0)

# time_diff_Dam_AutoClave의 값이 130 이상이면 1을 부여하는 파생 변수 생성
train_data['time_gap_All'] = train_data['time_diff_Dam_AutoClave'].apply(lambda x: 1 if x >= 130 else 0)
test_data['time_gap_All'] = test_data['time_diff_Dam_AutoClave'].apply(lambda x: 1 if x >= 130 else 0)

In [1224]:
# 제거할 변수 목록
columns_to_drop = [
    # 'Collect Date_Dam', 'Collect Date_AutoClave', 'Collect Date_Fill1', 'Collect Date_Fill2',
    'time_diff_Dam_Fill1', 'time_diff_Fill1_Fill2', 'time_diff_Fill2_AutoClave', 'time_diff_Dam_AutoClave'
]

# 변수 제거
train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

In [1225]:
train_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 105 columns):
 #    Column                                                 Dtype         
---   ------                                                 -----         
 0    model_suffix                                           object        
 1    Collect Date_Dam                                       datetime64[ns]
 2    CURE SPEED Collect Result_Dam                          int64         
 3    CURE STANDBY POSITION X Collect Result_Dam             int64         
 4    CURE STANDBY POSITION Z Collect Result_Dam             float64       
 5    CURE STANDBY POSITION Θ Collect Result_Dam             int64         
 6    DISCHARGED SPEED OF RESIN Collect Result_Dam           int64         
 7    HEAD Standby Position X Collect Result_Dam             float64       
 8    HEAD Standby Position Y Collect Result_Dam             int64         
 9    HEAD Standby Position Z Collect Result_Dam      

### 11. 변수 확인

In [1226]:
# 삭제할 변수 리스트
columns_to_drop = [
    'Chamber Temp. Judge Value_AutoClave', 
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave', 
    'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave'
]

train_data = train_data.drop(columns=columns_to_drop)
test_data = test_data.drop(columns=columns_to_drop)

In [1227]:
# 값의 종류가 1개이고 결측값이 없는 열을 제거하는 함수
def drop_single_value_columns(df):
    cols_to_drop = [col for col in df.columns if col != 'target' and df[col].nunique() == 1 and df[col].isnull().sum() == 0]
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped, cols_to_drop

# train_data와 test_data에서 해당 열 제거 및 삭제된 열 이름과 개수 출력
train_data, train_cols_dropped = drop_single_value_columns(train_data)
test_data, test_cols_dropped = drop_single_value_columns(test_data)

# print("삭제된 train_data 열 이름:", train_cols_dropped)
print("삭제된 train_data 열 개수:", len(train_cols_dropped))

# print("삭제된 test_data 열 이름:", test_cols_dropped)
print("삭제된 test_data 열 개수:", len(test_cols_dropped))

삭제된 train_data 열 개수: 37
삭제된 test_data 열 개수: 37


### 12. target encoding

In [1228]:
# object 타입의 변수 출력
train_object_columns = train_data.select_dtypes(include=['object']).columns
test_object_columns = test_data.select_dtypes(include=['object']).columns

print(train_object_columns, f" train_object_columns 갯수 : {len(train_object_columns)}")
print(test_object_columns, f" test_object_columns 갯수 : {len(test_object_columns)}")

# 각 object 변수의 고유 값 개수 출력
print("\nTrain Data:")
for col in train_object_columns:
    unique_count = train_data[col].nunique()
    print(f"{col} unique 값 갯수: {unique_count}")

print("\nTest Data:")
for col in test_object_columns:
    unique_count = test_data[col].nunique()
    print(f"{col} unique 값 갯수: {unique_count}")

Index(['model_suffix', 'target', 'cleaned_workorder',
       'PalletID_Collect_Result', 'Production_Qty_Collect_Result',
       'cure_end_position_XZ_Fill2', 'cure_start_position_XZ_Fill2',
       'cure_standby_position_XZ_Fill2', 'stage1_line13_distance_speed_Dam',
       'stage1_line24_distance_speed_Dam', 'stage2_line13_distance_speed_Dam',
       'stage2_line24_distance_speed_Dam', 'stage3_line13_distance_speed_Dam',
       'stage3_line24_distance_speed_Dam'],
      dtype='object')  train_object_columns 갯수 : 14
Index(['Set ID', 'model_suffix', 'cleaned_workorder',
       'PalletID_Collect_Result', 'Production_Qty_Collect_Result',
       'cure_end_position_XZ_Fill2', 'cure_start_position_XZ_Fill2',
       'cure_standby_position_XZ_Fill2', 'stage1_line13_distance_speed_Dam',
       'stage1_line24_distance_speed_Dam', 'stage2_line13_distance_speed_Dam',
       'stage2_line24_distance_speed_Dam', 'stage3_line13_distance_speed_Dam',
       'stage3_line24_distance_speed_Dam'],
      dtyp

In [1229]:
# 전체 평균 타겟 값 계산 (abnormal 전체 비율)
train_data['target_01'] = train_data['target'].apply(lambda x: 1 if x == 'AbNormal' else 0)
global_mean = train_data['target_01'].mean()

In [1230]:
# 적용할 열 리스트
columns_to_encode = [
    'model_suffix',
    'cleaned_workorder',
    'PalletID_Collect_Result',
    'cure_end_position_XZ_Fill2',
    'cure_start_position_XZ_Fill2',
    'cure_standby_position_XZ_Fill2',
    'stage1_line13_distance_speed_Dam',
    'stage1_line24_distance_speed_Dam',
    'stage2_line13_distance_speed_Dam',
    'stage2_line24_distance_speed_Dam',
    'stage3_line13_distance_speed_Dam',
    'stage3_line24_distance_speed_Dam',

    # 추가로 Production_Qty_Collect_Result도 인코딩
    'Production_Qty_Collect_Result'
]

# 전체 데이터의 평균 타겟값
global_mean = train_data['target_01'].mean()

for column in columns_to_encode:
    # 각 column에 대한 평균 타겟값과 카운트 계산
    target_mean = train_data.groupby(column)['target_01'].mean()
    count = train_data.groupby(column)['target_01'].count()

    # 스무딩 적용
    '''
    추천 알파 값:
    0.5: 일반적으로 많이 사용되는 값으로, 기존 데이터와 전체 평균 간의 균형을 잘 맞춰줍니다.
    0.3: 데이터가 충분히 많고 각 카테고리의 타겟 값이 잘 분포되어 있을 때 사용.
    0.7: 데이터가 적거나 특정 카테고리가 상대적으로 적을 때 사용.
    '''
    alpha = 0.5
    smoothed_values = (target_mean * count + global_mean * alpha) / (count + alpha)

    # 인코딩된 값을 데이터프레임에 추가
    train_data[f'{column}_encoded'] = train_data[column].map(smoothed_values)

    # test_data에 동일한 인코딩 값을 추가
    encoding_dict = train_data.groupby(column)[f'{column}_encoded'].first().to_dict()
    test_data[f'{column}_encoded'] = test_data[column].map(encoding_dict)

In [1231]:
# 삭제할 열 리스트
columns_to_drop = [
    'target_01',
    'model_suffix',
    'cleaned_workorder',
    'PalletID_Collect_Result',
    'cure_end_position_XZ_Fill2',
    'cure_start_position_XZ_Fill2',
    'cure_standby_position_XZ_Fill2',
    'stage1_line13_distance_speed_Dam',
    'stage1_line24_distance_speed_Dam',
    'stage2_line13_distance_speed_Dam',
    'stage2_line24_distance_speed_Dam',
    'stage3_line13_distance_speed_Dam',
    'stage3_line24_distance_speed_Dam',

    'Production_Qty_Collect_Result'
]

# train_data와 test_data에서 열 드랍
train_data = train_data.drop(columns=columns_to_drop, errors='ignore')
test_data = test_data.drop(columns=columns_to_drop, errors='ignore')

In [1232]:
# target 열을 제외한 나머지 열의 결측치를 0.05로 채우기
test_data.fillna(value={col: 0.05 for col in test_data.columns if col != 'target'}, inplace=True)

In [1233]:
# info 잘리지 않게 출력
train_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 65 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Collect Date_Dam                                40506 non-null  datetime64[ns]
 1   CURE SPEED Collect Result_Dam                   40506 non-null  int64         
 2   DISCHARGED SPEED OF RESIN Collect Result_Dam    40506 non-null  int64         
 3   Head Clean Position Z Collect Result_Dam        40506 non-null  float64       
 4   Head Purge Position Z Collect Result_Dam        40506 non-null  float64       
 5   Head Zero Position Y Collect Result_Dam         40506 non-null  float64       
 6   Stage1_Circle_Distance_Speed_Dam                40506 non-null  int64         
 7   Stage2_Circle_Distance_Speed_Dam                40506 non-null  int64         
 8   Stage3_Circle_Distance_Speed_Dam              

In [1234]:
# info 잘리지 않게 출력
test_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 66 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Set ID                                          17361 non-null  object        
 1   Collect Date_Dam                                17361 non-null  datetime64[ns]
 2   CURE SPEED Collect Result_Dam                   17361 non-null  int64         
 3   DISCHARGED SPEED OF RESIN Collect Result_Dam    17361 non-null  int64         
 4   Head Clean Position Z Collect Result_Dam        17361 non-null  float64       
 5   Head Purge Position Z Collect Result_Dam        17361 non-null  float64       
 6   Head Zero Position Y Collect Result_Dam         17361 non-null  float64       
 7   Stage1_Circle_Distance_Speed_Dam                17361 non-null  int64         
 8   Stage2_Circle_Distance_Speed_Dam              

### 13. correlation

In [1235]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result_encoded',
    'WorkMode Collect Result',
    'Receip_n_suffix_3',
    'time_gap_All'
]

In [1236]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

- dam

In [1237]:
# 상관관계를 확인할 데이터셋
combined_variables = var_dam_fill + var_all_corr + [var for var in train_data.columns if '_Dam' in var]
combined_variables

['Equipment_same_num',
 'PalletID_Collect_Result_encoded',
 'Production_Qty_Collect_Result_encoded',
 'WorkMode Collect Result',
 'Receip_n_suffix_3',
 'time_gap_All',
 'model_suffix_encoded',
 'cleaned_workorder_encoded',
 'Collect Date_Dam',
 'CURE SPEED Collect Result_Dam',
 'DISCHARGED SPEED OF RESIN Collect Result_Dam',
 'Head Clean Position Z Collect Result_Dam',
 'Head Purge Position Z Collect Result_Dam',
 'Head Zero Position Y Collect Result_Dam',
 'Stage1_Circle_Distance_Speed_Dam',
 'Stage2_Circle_Distance_Speed_Dam',
 'Stage3_Circle_Distance_Speed_Dam',
 'CURE_DISTANCE_Dam',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Dam',
 'HEAD NORMAL DISTANCE_TRIANGLE_area_Dam',
 'HEAD NORMAL DISTANCE_TRIANGLE_height_Dam',
 'volume_time_multip_stage1_Dam',
 'volume_time_multip_stage2_Dam',
 'volume_time_multip_stage3_Dam',
 'volume_time_multip_avg_Dam',
 'average_thickness_Dam',
 'time_ratio_Dam',
 'time_gap_Dam',
 'stage1_line13_distance_speed_Dam_encoded',
 'stage1_line24_distance_speed_Dam_

In [1238]:
variables = ['Equipment_same_num',
 'PalletID_Collect_Result_encoded',
 'Production_Qty_Collect_Result_encoded',
 'WorkMode Collect Result',
 'Receip_n_suffix_3',
 'time_gap_All',
 'CURE SPEED Collect Result_Dam',
 'DISCHARGED SPEED OF RESIN Collect Result_Dam',
 'Head Clean Position Z Collect Result_Dam',
 'Head Purge Position Z Collect Result_Dam',
 'Head Zero Position Y Collect Result_Dam',
 #'Stage1_Circle_Distance_Speed_Dam',
 'Stage2_Circle_Distance_Speed_Dam',
 #'Stage3_Circle_Distance_Speed_Dam',
 'CURE_DISTANCE_Dam',
 #'HEAD NORMAL DISTANCE_TRIANGLE_area_Dam',
 'HEAD NORMAL DISTANCE_TRIANGLE_height_Dam',
 'volume_time_multip_stage1_Dam',
 'volume_time_multip_stage2_Dam',
 #'volume_time_multip_stage3_Dam',
 #'volume_time_multip_avg_Dam',
 'average_thickness_Dam',
 'time_ratio_Dam',
  'time_gap_Dam',
 #'stage1_line13_distance_speed_Dam_encoded',
 'stage1_line24_distance_speed_Dam_encoded',
 #'stage2_line13_distance_speed_Dam_encoded',
 'stage2_line24_distance_speed_Dam_encoded',
 #'stage3_line13_distance_speed_Dam_encoded',
 #'stage3_line24_distance_speed_Dam_encoded',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Dam',
 'time_gap_Dam'
 ]

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [1239]:
# 자기자신을 제외하고 상관관계 절댓값이 0.9 이상인 조합 찾기
correlation_matrix = filtered_data.corr()
strong_correlations = correlation_matrix[(correlation_matrix.abs() >= 0.9) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [1240]:
# 드랍할 열 목록
columns_to_drop = [
    'Stage1_Circle_Distance_Speed_Dam',
    'Stage3_Circle_Distance_Speed_Dam',
    'HEAD NORMAL DISTANCE_TRIANGLE_area_Dam',
    'stage1_line24_distance_speed_Dam_encoded',
    'stage2_line13_distance_speed_Dam_encoded',
    'stage3_line13_distance_speed_Dam_encoded',
    'stage3_line24_distance_speed_Dam_encoded',
    'volume_time_multip_stage3_Dam',
    'volume_time_multip_avg_Dam'
]

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [1241]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [1242]:
train_data_dam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 26 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Equipment_same_num                            40506 non-null  int64         
 1   PalletID_Collect_Result_encoded               40506 non-null  float64       
 2   Production_Qty_Collect_Result_encoded         40506 non-null  float64       
 3   WorkMode Collect Result                       40506 non-null  float64       
 4   Receip_n_suffix_3                             40506 non-null  int64         
 5   time_gap_All                                  40506 non-null  int64         
 6   target                                        40506 non-null  object        
 7   model_suffix_encoded                          40506 non-null  float64       
 8   cleaned_workorder_encoded                     40506 non-null  floa

In [1243]:
test_data_dam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 27 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Equipment_same_num                            17361 non-null  int64         
 1   PalletID_Collect_Result_encoded               17361 non-null  float64       
 2   Production_Qty_Collect_Result_encoded         17361 non-null  float64       
 3   WorkMode Collect Result                       17361 non-null  float64       
 4   Receip_n_suffix_3                             17361 non-null  int64         
 5   time_gap_All                                  17361 non-null  int64         
 6   Set ID                                        17361 non-null  object        
 7   target                                        0 non-null      float64       
 8   model_suffix_encoded                          17361 non-null  floa

- fill1

In [1244]:
# 상관관계를 확인할 데이터셋
combined_variables = var_dam_fill + var_all_corr + [var for var in train_data.columns if '_Fill1' in var]
combined_variables

['Equipment_same_num',
 'PalletID_Collect_Result_encoded',
 'Production_Qty_Collect_Result_encoded',
 'WorkMode Collect Result',
 'Receip_n_suffix_3',
 'time_gap_All',
 'model_suffix_encoded',
 'cleaned_workorder_encoded',
 'Collect Date_Fill1',
 'DISCHARGED SPEED OF RESIN Collect Result_Fill1',
 'Head Purge Position Z Collect Result_Fill1',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill1',
 'HEAD NORMAL DISTANCE_TRIANGLE_area_Fill1',
 'HEAD NORMAL DISTANCE_TRIANGLE_height_Fill1',
 'volume_time_multip_stage1_Fill1',
 'volume_time_multip_stage2_Fill1',
 'volume_time_multip_stage3_Fill1',
 'volume_time_multip_avg_Fill1',
 'time_ratio_Fill1',
 'time_gap_Fill1']

In [1245]:
variables = ['Equipment_same_num',
 'PalletID_Collect_Result_encoded',
 'Production_Qty_Collect_Result_encoded',
 'WorkMode Collect Result',
 'Receip_n_suffix_3',
 'time_gap_All',
 'model_suffix_encoded',
 'cleaned_workorder_encoded',
 'DISCHARGED SPEED OF RESIN Collect Result_Fill1',
 'Head Purge Position Z Collect Result_Fill1',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill1',
 #'HEAD NORMAL DISTANCE_TRIANGLE_area_Fill1',
 'HEAD NORMAL DISTANCE_TRIANGLE_height_Fill1',
 #'volume_time_multip_avg_Fill1',
 'volume_time_multip_stage1_Fill1',
 #'volume_time_multip_stage2_Fill1',
 #'volume_time_multip_stage3_Fill1',
 'time_ratio_Fill1',
  'time_gap_Fill1']

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [1246]:
# 자기자신을 제외하고 상관관계 절댓값이 0.9 이상인 조합 찾기
correlation_matrix = filtered_data.corr()
strong_correlations = correlation_matrix[(correlation_matrix.abs() >= 0.9) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [1247]:
# 드랍할 열 목록
columns_to_drop = [
    'HEAD NORMAL DISTANCE_TRIANGLE_area_Fill1',
    'volume_time_multip_avg_Fill1',
    'volume_time_multip_stage2_Fill1',
    'volume_time_multip_stage3_Fill1'
]

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [1248]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [1249]:
train_data_fill1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 17 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Equipment_same_num                              40506 non-null  int64         
 1   PalletID_Collect_Result_encoded                 40506 non-null  float64       
 2   Production_Qty_Collect_Result_encoded           40506 non-null  float64       
 3   WorkMode Collect Result                         40506 non-null  float64       
 4   Receip_n_suffix_3                               40506 non-null  int64         
 5   time_gap_All                                    40506 non-null  int64         
 6   target                                          40506 non-null  object        
 7   model_suffix_encoded                            40506 non-null  float64       
 8   cleaned_workorder_encoded                     

In [1250]:
test_data_fill1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 18 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Equipment_same_num                              17361 non-null  int64         
 1   PalletID_Collect_Result_encoded                 17361 non-null  float64       
 2   Production_Qty_Collect_Result_encoded           17361 non-null  float64       
 3   WorkMode Collect Result                         17361 non-null  float64       
 4   Receip_n_suffix_3                               17361 non-null  int64         
 5   time_gap_All                                    17361 non-null  int64         
 6   Set ID                                          17361 non-null  object        
 7   target                                          0 non-null      float64       
 8   model_suffix_encoded                          

- fill2

In [1251]:
# 상관관계를 확인할 데이터셋
combined_variables = var_dam_fill + var_all_corr + [var for var in train_data.columns if '_Fill2' in var]
combined_variables

['Equipment_same_num',
 'PalletID_Collect_Result_encoded',
 'Production_Qty_Collect_Result_encoded',
 'WorkMode Collect Result',
 'Receip_n_suffix_3',
 'time_gap_All',
 'model_suffix_encoded',
 'cleaned_workorder_encoded',
 'Collect Date_Fill2',
 'CURE SPEED Collect Result_Fill2',
 'Head Purge Position Z Collect Result_Fill2',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE2_Fill2',
 'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill2',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill2',
 'time_ratio_Fill2',
 'time_gap_Fill2',
 'cure_end_position_XZ_Fill2_encoded',
 'cure_start_position_XZ_Fill2_encoded',
 'cure_standby_position_XZ_Fill2_encoded']

In [1252]:
variables = [
    'Equipment_same_num',
 'PalletID_Collect_Result_encoded',
 'Production_Qty_Collect_Result_encoded',
 'WorkMode Collect Result',
 'Receip_n_suffix_3',
 'time_gap_All',
 'model_suffix_encoded',
 'cleaned_workorder_encoded',
 'CURE SPEED Collect Result_Fill2',
 'Head Purge Position Z Collect Result_Fill2',
 'HEAD NORMAL DISTANCE_STAGE1_STAGE2_Fill2',
 #'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill2',
 #'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill2',
 'time_ratio_Fill2',
 'time_gap_Fill2',
 'cure_end_position_XZ_Fill2_encoded',
 'cure_start_position_XZ_Fill2_encoded']
 #'cure_standby_position_XZ_Fill2_encoded']

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [1253]:
# 자기자신을 제외하고 상관관계 절댓값이 0.9 이상인 조합 찾기
correlation_matrix = filtered_data.corr()
strong_correlations = correlation_matrix[(correlation_matrix.abs() >= 0.9) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [1254]:
# 드랍할 열 목록
columns_to_drop = [
    'HEAD NORMAL DISTANCE_STAGE2_STAGE3_Fill2',
    'HEAD NORMAL DISTANCE_STAGE1_STAGE3_Fill2',
    'cure_standby_position_XZ_Fill2_encoded'
]

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [1255]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [1256]:
train_data_fill2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Equipment_same_num                          40506 non-null  int64         
 1   PalletID_Collect_Result_encoded             40506 non-null  float64       
 2   Production_Qty_Collect_Result_encoded       40506 non-null  float64       
 3   WorkMode Collect Result                     40506 non-null  float64       
 4   Receip_n_suffix_3                           40506 non-null  int64         
 5   time_gap_All                                40506 non-null  int64         
 6   target                                      40506 non-null  object        
 7   model_suffix_encoded                        40506 non-null  float64       
 8   cleaned_workorder_encoded                   40506 non-null  float64       
 9   Collec

In [1257]:
test_data_fill2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Equipment_same_num                          17361 non-null  int64         
 1   PalletID_Collect_Result_encoded             17361 non-null  float64       
 2   Production_Qty_Collect_Result_encoded       17361 non-null  float64       
 3   WorkMode Collect Result                     17361 non-null  float64       
 4   Receip_n_suffix_3                           17361 non-null  int64         
 5   time_gap_All                                17361 non-null  int64         
 6   Set ID                                      17361 non-null  object        
 7   target                                      0 non-null      float64       
 8   model_suffix_encoded                        17361 non-null  float64       
 9   cleane

- autoclave

In [1258]:
# 상관관계를 확인할 데이터셋
combined_variables = var_all_corr + [var for var in train_data.columns if '_AutoClave' in var]
combined_variables

['model_suffix_encoded',
 'cleaned_workorder_encoded',
 'Collect Date_AutoClave',
 'Chamber Temp. Collect Result_AutoClave',
 '1st_pressure_time_AutoClave',
 '2nd_pressure_time_AutoClave',
 '3rd_pressure_time_AutoClave',
 'avg_pressure_time_AutoClave',
 'time_ratio_AutoClave']

In [1259]:
variables = ['model_suffix_encoded',
 'cleaned_workorder_encoded',
 'Chamber Temp. Collect Result_AutoClave',
 '1st_pressure_time_AutoClave',
 '2nd_pressure_time_AutoClave',
 '3rd_pressure_time_AutoClave',
 #'avg_pressure_time_AutoClave',
 'time_ratio_AutoClave']

# 변수들로만 이루어진 DataFrame 생성
filtered_data = train_data[variables]

In [1260]:
# 자기자신을 제외하고 상관관계 절댓값이 0.9 이상인 조합 찾기
correlation_matrix = filtered_data.corr()
strong_correlations = correlation_matrix[(correlation_matrix.abs() >= 0.9) & (correlation_matrix != 1)]

# 리스트로 변환
strong_correlations_pairs = strong_correlations.stack().reset_index()
strong_correlations_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']

# 결과 출력
print(strong_correlations_pairs)

Empty DataFrame
Columns: [Variable 1, Variable 2, Correlation]
Index: []


In [1261]:
# 드랍할 열 목록
columns_to_drop = ['avg_pressure_time_AutoClave']

# 열 삭제
train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [1262]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [1263]:
plus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   day                                       105 non-null    int64  
 1   start_time                                105 non-null    object 
 2   end_time                                  105 non-null    object 
 3   Dam Thickness - 1time / day               81 non-null     float64
 4   Fill thickness - 1time / day              81 non-null     float64
 5   Fill cure energy (mJ) - Every break time  81 non-null     float64
 6   Fill cure power (mW) - Every break time   81 non-null     float64
 7   Line Temp. - Every break time             81 non-null     float64
 8   Line humidity - Every break time          81 non-null     float64
 9   OCR gap - Line#1 stage1                   81 non-null     float64
 10  OCR gap - Line#1 stage2               

In [1264]:
plus_data['day']

0      240319
1      240319
2      240319
3      240319
4      240319
        ...  
100    240428
101    240428
102    240428
103    240428
104    240428
Name: day, Length: 105, dtype: int64

In [1265]:
plus_data['start_time']

0      08:00:00
1      10:45:00
2      13:00:00
3      15:15:00
4      18:30:00
         ...   
100    08:00:00
101    10:45:00
102    13:00:00
103    15:15:00
104    18:30:00
Name: start_time, Length: 105, dtype: object

In [1266]:
import pandas as pd

# 'start_time' 열을 datetime 형식으로 변환하고 'HHMMSS' 형식으로 저장
plus_data['start_time'] = pd.to_datetime(plus_data['start_time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H%M%S')

# 결과 확인
print(plus_data['start_time'].head())

0    080000
1    104500
2    130000
3    151500
4    183000
Name: start_time, dtype: object


In [1267]:
import pandas as pd

# 'start_time' 열을 datetime 형식으로 변환하고 'HHMMSS' 형식으로 저장
plus_data['end_time'] = pd.to_datetime(plus_data['end_time'], format='%H:%M:%S', errors='coerce').dt.strftime('%H%M%S')

# 결과 확인
print(plus_data['end_time'].head())

0    103000
1    120000
2    150000
3    180000
4    200000
Name: end_time, dtype: object


In [1268]:
import pandas as pd

# 'Collect Date_Dam' 열을 datetime 형식으로 변환
train_data_dam['Collect Date_Dam'] = pd.to_datetime(train_data_dam['Collect Date_Dam'], errors='coerce')

# 'Collect Date_Dam' 열에서 날짜와 시간을 분리하여 각각 'day'와 'time' 열로 저장
train_data_dam['day'] = train_data_dam['Collect Date_Dam'].dt.strftime('%y%m%d')
train_data_dam['time'] = train_data_dam['Collect Date_Dam'].dt.strftime('%H%M%S')

# 결과 확인
print(train_data_dam[['Collect Date_Dam', 'day', 'time']].head())

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


     Collect Date_Dam     day    time
0 2024-04-25 11:10:00  240425  111000
1 2023-09-19 14:30:00  230919  143000
2 2024-03-05 09:30:00  240305  093000
3 2023-09-25 15:40:00  230925  154000
4 2023-06-27 13:20:00  230627  132000


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1269]:
train_data_dam['day'] = train_data_dam['day'].astype(int)

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1270]:
train_data_dam['time'] = train_data_dam['time'].astype(int)

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1271]:
plus_data['start_time'] = plus_data['start_time'].astype(int)

In [1272]:
plus_data['end_time'] = plus_data['end_time'].astype(int)

In [1273]:
plus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   day                                       105 non-null    int64  
 1   start_time                                105 non-null    int32  
 2   end_time                                  105 non-null    int32  
 3   Dam Thickness - 1time / day               81 non-null     float64
 4   Fill thickness - 1time / day              81 non-null     float64
 5   Fill cure energy (mJ) - Every break time  81 non-null     float64
 6   Fill cure power (mW) - Every break time   81 non-null     float64
 7   Line Temp. - Every break time             81 non-null     float64
 8   Line humidity - Every break time          81 non-null     float64
 9   OCR gap - Line#1 stage1                   81 non-null     float64
 10  OCR gap - Line#1 stage2               

In [1274]:
plus_data['start_time']

0       80000
1      104500
2      130000
3      151500
4      183000
        ...  
100     80000
101    104500
102    130000
103    151500
104    183000
Name: start_time, Length: 105, dtype: int32

In [1275]:
# train_data_dam의 각 행에 대해 plus_data의 day와 time 구간에 해당하는 값을 확인
def check_and_concat(row):
    matching_rows = plus_data[(plus_data['day'] == row['day']) & 
                              (plus_data['start_time'] <= row['time']) & 
                              (plus_data['end_time'] >= row['time'])]
    if not matching_rows.empty:
        return pd.concat([row.to_frame().T] * len(matching_rows), axis=1).join(matching_rows.reset_index(drop=True))
    else:
        return pd.DataFrame()

# 각 행에 대해 check_and_concat 함수 적용
result = train_data_dam.apply(check_and_concat, axis=1)

# 결과를 하나의 데이터프레임으로 결합
final_result = pd.concat(result.tolist(), ignore_index=True)

# 결과 확인
print(final_result)

ValueError: columns overlap but no suffix specified: Index(['day'], dtype='object')

In [1161]:
plus_data['day'].value_counts()

240319    5
240405    5
240425    5
240423    5
240419    5
240417    5
240411    5
240410    5
240409    5
240406    5
240404    5
240320    5
240402    5
240401    5
240330    5
240329    5
240328    5
240327    5
240326    5
240321    5
240428    5
Name: day, dtype: int64

In [1162]:
train_data_dam['day'].value_counts()

231220    407
231218    399
240106    377
231219    368
240108    368
         ... 
231026     13
240416      9
230824      6
230504      4
230815      3
Name: day, Length: 210, dtype: int64

In [1163]:
train_data_dam['day']

0        240425
1        230919
2        240305
3        230925
4        230627
          ...  
40501    230914
40502    240410
40503    240222
40504    230725
40505    230531
Name: day, Length: 40506, dtype: int32

In [1166]:
# plus_data['day']와 train_data_dam['day']로 그룹화하여 출력
plus_data_grouped = plus_data.groupby('day').size()
train_data_dam_grouped = train_data_dam.groupby('day').size()

print("plus_data 그룹화 결과:")
print(plus_data_grouped)

print("\ntrain_data_dam 그룹화 결과:")
print(train_data_dam_grouped)

# 같은 값이 있는지 비교
common_days = set(plus_data['day']).intersection(set(train_data_dam['day']))

print("\n같은 값이 있는 day:")
print(common_days)

plus_data 그룹화 결과:
day
240319    5
240320    5
240321    5
240326    5
240327    5
240328    5
240329    5
240330    5
240401    5
240402    5
240404    5
240405    5
240406    5
240409    5
240410    5
240411    5
240417    5
240419    5
240423    5
240425    5
240428    5
dtype: int64

train_data_dam 그룹화 결과:
day
230504      4
230505    102
230506    147
230510    140
230511    170
         ... 
240422    158
240423     59
240424     20
240425    157
240428    281
Length: 210, dtype: int64

같은 값이 있는 day:
{240401, 240402, 240404, 240405, 240406, 240409, 240410, 240411, 240417, 240419, 240423, 240425, 240428, 240319, 240320, 240321, 240326, 240327, 240328, 240329, 240330}


In [946]:
import pandas as pd

# train_data_dam의 각 행에 대해 plus_data의 day와 time 구간에 해당하는 값을 확인
def check_and_concat(row):
    matching_rows = plus_data[(plus_data['day'] == row['day']) ]
    if not matching_rows.empty:
        return pd.concat([row.to_frame().T] * len(matching_rows), axis=1).join(matching_rows.reset_index(drop=True))
    else:
        return pd.DataFrame()

# 각 행에 대해 check_and_concat 함수 적용
result = train_data_dam.apply(check_and_concat, axis=1)

# 결과를 하나의 데이터프레임으로 결합
final_result = pd.concat(result.tolist(), ignore_index=True)

# 결과 확인
print(final_result)

ValueError: columns overlap but no suffix specified: Index(['day'], dtype='object')

In [101]:
plus_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 17 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   day                                       105 non-null    datetime64[ns]
 1   start_time                                105 non-null    object        
 2   end_time                                  105 non-null    object        
 3   Dam Thickness - 1time / day               81 non-null     float64       
 4   Fill thickness - 1time / day              81 non-null     float64       
 5   Fill cure energy (mJ) - Every break time  81 non-null     float64       
 6   Fill cure power (mW) - Every break time   81 non-null     float64       
 7   Line Temp. - Every break time             81 non-null     float64       
 8   Line humidity - Every break time          81 non-null     float64       
 9   OCR gap - Line#1 stage1         

In [102]:
plus_data['start_datatime']

0     2024-03-19 08:00:00
1     2024-03-19 10:45:00
2     2024-03-19 13:00:00
3     2024-03-19 15:15:00
4     2024-03-19 18:30:00
              ...        
100   2024-04-28 08:00:00
101   2024-04-28 10:45:00
102   2024-04-28 13:00:00
103   2024-04-28 15:15:00
104   2024-04-28 18:30:00
Name: start_datatime, Length: 105, dtype: datetime64[ns]

In [103]:
train_data_dam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 26 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   Equipment_same_num                            40506 non-null  int64         
 1   PalletID_Collect_Result_encoded               40506 non-null  float64       
 2   Production_Qty_Collect_Result_encoded         40506 non-null  float64       
 3   WorkMode Collect Result                       40506 non-null  float64       
 4   Receip_n_suffix_3                             40506 non-null  int64         
 5   time_gap_All                                  40506 non-null  int64         
 6   target                                        40506 non-null  object        
 7   model_suffix_encoded                          40506 non-null  float64       
 8   cleaned_workorder_encoded                     40506 non-null  floa

In [822]:
train_data_dam['Collect Date_Dam']

0       2024-04-25 11:10:00
1       2023-09-19 14:30:00
2       2024-03-05 09:30:00
3       2023-09-25 15:40:00
4       2023-06-27 13:20:00
                ...        
40501   2023-09-14 15:50:00
40502   2024-04-10 14:20:00
40503   2024-02-22 08:40:00
40504   2023-07-25 11:00:00
40505   2023-05-31 01:40:00
Name: Collect Date_Dam, Length: 40506, dtype: datetime64[ns]

In [105]:
plus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 17 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   day                                       105 non-null    datetime64[ns]
 1   start_time                                105 non-null    object        
 2   end_time                                  105 non-null    object        
 3   Dam Thickness - 1time / day               81 non-null     float64       
 4   Fill thickness - 1time / day              81 non-null     float64       
 5   Fill cure energy (mJ) - Every break time  81 non-null     float64       
 6   Fill cure power (mW) - Every break time   81 non-null     float64       
 7   Line Temp. - Every break time             81 non-null     float64       
 8   Line humidity - Every break time          81 non-null     float64       
 9   OCR gap - Line#1 stage1         

In [107]:
import pandas as pd

# 'start_datetime'과 'end_datetime' 열을 datetime 형식으로 변환 (이미 datetime 형식일 경우에도 안전하게 변환)
plus_data['start_datetime'] = pd.to_datetime(plus_data['start_datetime'], errors='coerce')
plus_data['end_datetime'] = pd.to_datetime(plus_data['end_datetime'], errors='coerce')

# 'Collect Date_Dam' 열을 datetime 형식으로 변환
train_data_dam['Collect Date_Dam'] = pd.to_datetime(train_data_dam['Collect Date_Dam'], errors='coerce')

# train_data_dam의 각 행에 대해 plus_data의 start_datetime과 end_datetime 사이에 해당하는 값을 확인
def check_and_concat(row):
    mask = (plus_data['start_datetime'] <= row['Collect Date_Dam']) & (plus_data['end_datetime'] >= row['Collect Date_Dam'])
    matching_rows = plus_data[mask]
    if not matching_rows.empty:
        return pd.concat([row.to_frame().T] * len(matching_rows), axis=1).join(matching_rows.reset_index(drop=True))
    else:
        return row.to_frame().T

# 각 행에 대해 check_and_concat 함수 적용
result = train_data_dam.apply(check_and_concat, axis=1)

# 결과를 하나의 데이터프레임으로 결합
final_result = pd.concat(result.tolist(), ignore_index=True)

# 결과 확인
print(final_result.head())

KeyError: 'start_datetime'

In [95]:
# # DataFrame을 CSV 파일로 저장
# train_data.to_csv('./final_data/final_train_data_ver2.csv', index=False)
# test_data.to_csv('./final_data/final_test_data_ver2.csv', index=False)

## 모델링

In [96]:
import numpy as np
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기

In [97]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("./final_data/final_train_data_ver2.csv")
test_data = pd.read_csv("./final_data/final_test_data_ver2.csv")

In [98]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result_encoded',
    'WorkMode Collect Result',
    'Receip_n_suffix_3',
    'time_gap_All'
]

In [99]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [100]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [101]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [102]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [103]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [105]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 44
train_data_dam DataFrame의 칼럼 수: 25
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 16
train_data_fill2 DataFrame의 칼럼 수: 16
----test data-----
test_data DataFrame의 칼럼 수: 45
test_data_dam DataFrame의 칼럼 수: 26
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 17
test_data_fill2 DataFrame의 칼럼 수: 17


In [109]:
train_data_autoclave.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   target                                  40506 non-null  object 
 1   model_suffix_encoded                    40506 non-null  float64
 2   cleaned_workorder_encoded               40506 non-null  float64
 3   Chamber Temp. Collect Result_AutoClave  40506 non-null  int64  
 4   1st_pressure_time_AutoClave             40506 non-null  float64
 5   2nd_pressure_time_AutoClave             40506 non-null  float64
 6   3rd_pressure_time_AutoClave             40506 non-null  float64
 7   time_ratio_AutoClave                    40506 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 2.5+ MB


---

## 모델링

### 모델 정의

In [961]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

### 공정별 모델 구축

### lgbm

In [102]:
model_Dam = fit_all_train_data_function(
    'lgbm', train_data_dam
    , n_estimators=2470
    , num_leaves=2454
    , max_depth=26
    , learning_rate=0.06067228197373452
    , min_child_samples=134
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_AutoClave = fit_all_train_data_function(
    'lgbm', train_data_autoclave
    , n_estimators=731
    , num_leaves=996
    , max_depth=273
    , learning_rate=0.0912254393922836
    , min_child_samples=195
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=821
    , num_leaves=1400
    , max_depth=52
    , learning_rate=0.002743887584386348
    , min_child_samples=231
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=1005
    , num_leaves=2304
    , max_depth=293
    , learning_rate=0.08460539739469425
    , min_child_samples=272
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_All = fit_all_train_data_function(
    'lgbm', train_data
    , n_estimators=1496
    , num_leaves=1611
    , max_depth=148
    , learning_rate=0.0822880159816304
    , min_child_samples=194
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)


# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
lgbm_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

lgbm 모델이 train_data_dam 데이터로 학습 완료
lgbm 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료
lgbm 모델이 train_data 데이터로 학습 완료


In [ ]:
# 소프트 보팅 결과
final_predictions = voting(lgbm_probs, method='soft', threshold=0.26)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data09xx_lgbm.csv", index=False)

### xgb

In [ ]:
model_Dam = fit_all_train_data_function(
    'xgb', train_data_dam
    , n_estimators = 1244
    , learning_rate = 0.1258535425769987
    , max_depth = 26
    , alpha = 2.1820842842359597e-06
    , gamma = 0.00010809657684921935
    , reg_alpha = 0.5844029076359536
    , reg_lambda = 0.4748752246073433
    , colsample_bytree = 0.9607659760060685
    , subsample = 0.7147741317935203
    , objective = 'binary:logistic'
    , tree_method = 'exact'
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'xgb', train_data_autoclave,
    n_estimators = 1152, 
    learning_rate = 0.02466611382982541, 
    max_depth = 29, 
    alpha = 2.9180083404308157e-05, 
    gamma = 0.00012667501319666823, 
    reg_alpha = 0.6903592486292155, 
    reg_lambda = 0.5638873235014423, 
    colsample_bytree = 0.9432782030604233, 
    subsample = 0.19192246128663584,
    objective = 'binary:logistic',  # 이진 분류
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'xgb', train_data_fill1,
    n_estimators = 1899, 
    learning_rate = 0.011878583548993711, 
    max_depth = 12, 
    alpha = 0.004515243354832891,
    gamma = 0.0015693650802180896,
    reg_alpha = 0.7484424912256998, 
    reg_lambda = 0.27164326303977143, 
    colsample_bytree = 0.7901385059430825,
    subsample = 0.9924662032617025,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'xgb', train_data_fill2,
    n_estimators = 1162, 
    learning_rate = 0.014523070494025153, 
    max_depth = 8, 
    alpha = 0.00012198482017902725, 
    gamma = 0.001236902841680112, 
    reg_alpha = 0.7331637000614692, 
    reg_lambda = 0.5237223061096699, 
    colsample_bytree = 0.8250374170841293, 
    subsample = 0.31906427054137687,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'xgb', train_data,
    n_estimators = 2427,
    learning_rate = 0.010774204513905965, 
    max_depth = 17, 
    alpha = 0.0005233654110538582, 
    gamma = 5.551445919277608e-05, 
    reg_alpha = 0.9652805882189326, 
    reg_lambda = 0.3542856398135083, 
    colsample_bytree = 0.9094884645797131, 
    subsample = 0.1733751790853043,
    objective = 'binary:logistic',  # 이진 분류
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)


# 예측 확률 리스트 (소프트 보팅용)
xgb_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]


In [ ]:
# 소프트 보팅 결과
final_predictions = voting(xgb_probs, method='soft', threshold=0.24)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data09xx_xgb.csv", index=False)

### cat

In [ ]:
model_Dam = fit_all_train_data_function(
    'cat', train_data_dam,
    iterations = 1478, 
    learning_rate = 0.009068953796649421, 
    depth = 11, 
    min_data_in_leaf = 2,
    l2_leaf_reg = 1.187291687951122,
    random_strength = 0.43102541391012816, 
    bagging_temperature = 3.1790702578164853, 
    border_count = 155, 
    scale_pos_weight = 1.4418307437388553,
    grow_policy = 'Depthwise',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_AutoClave = fit_all_train_data_function(
    'cat', train_data_autoclave,
    iterations = 1299, 
    learning_rate =  0.03808793470493637, 
    depth = 9, 
    min_data_in_leaf = 5,
    l2_leaf_reg = 4.942829707223811, 
    random_strength = 3.804933757402697, 
    bagging_temperature = 1.3151583440997139, 
    border_count = 286, 
    scale_pos_weight = 1.9749286362629779,
    grow_policy = 'SymmetricTree',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_Fill1 = fit_all_train_data_function(
    'cat', train_data_fill1,
    iterations = 2842, 
    learning_rate = 0.01099464761153367, 
    depth = 4, 
    min_data_in_leaf = 3,
    l2_leaf_reg = 3.7373183252945945, 
    random_strength = 9.3675281753561, 
    bagging_temperature = 4.750112155842117, 
    border_count = 160, 
    scale_pos_weight = 2.53860325765727,
    grow_policy = 'Lossguide',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_Fill2 = fit_all_train_data_function(
    'cat', train_data_fill2,
    iterations = 1458, 
    learning_rate = 0.004706507801075929, 
    depth = 13, 
    min_data_in_leaf = 4,
    l2_leaf_reg = 1.909987690181427, 
    random_strength = 9.047942432889677, 
    bagging_temperature = 3.545210494821586, 
    border_count = 300, 
    scale_pos_weight = 3.4781865667208467,
    grow_policy = 'Lossguide',


    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

model_All = fit_all_train_data_function(
    'cat', train_data,
    iterations=1349,
    learning_rate=0.012526639112437014,
    depth=9,
    min_data_in_leaf=4,
    l2_leaf_reg=2.245006704049574,
    random_strength=0.6922797458293842,
    bagging_temperature=8.230635636022027,
    border_count=211,
    scale_pos_weight=2.0709015241138236,
    grow_policy='Depthwise',
    
    random_state=RANDOM_STATE,
    eval_metric='F1',
    logging_level='Silent',
    boosting_type='Plain'
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
cat_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

In [ ]:
# 소프트 보팅 결과
final_predictions = voting(cat_probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data09xx_cat.csv", index=False)

### et

In [ ]:
model_Dam = fit_all_train_data_function(
    'et', train_data_dam
    , n_estimators = 2242
    , max_depth = 32
    , min_samples_split = 2
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'et', train_data_autoclave,
    n_estimators = 2708,
    max_depth = 41,
    min_samples_split = 8,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'et', train_data_fill1,
    n_estimators = 1520,
    max_depth = 30,
    min_samples_split = 2,
    min_samples_leaf = 1,
    criterion = 'entropy',
    bootstrap = False,
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'et', train_data_fill2
    , n_estimators = 1001
    , max_depth = 45
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'et', train_data
    , n_estimators = 2884
    , max_depth = 56
    , min_samples_split = 3
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
et_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

In [ ]:
# 소프트 보팅 결과
final_predictions = voting(et_probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data09xx_et.csv", index=False)

### rf

In [ ]:

model_Dam = fit_all_train_data_function(
    'rf', train_data_dam
    , n_estimators = 1330
    , max_depth = 36
    , min_samples_split = 6
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'rf', train_data_autoclave
    , n_estimators = 1103
    , max_depth = 36
    , min_samples_split = 8
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , bootstrap = False
    , random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'rf', train_data_fill1
    , n_estimators = 1861
    , max_depth = 91
    , min_samples_split = 7
    , min_samples_leaf = 5
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'rf', train_data_fill2
    , n_estimators = 2663
    , max_depth = 100
    , min_samples_split = 6
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'rf', train_data
    , n_estimators = 1082
    , max_depth = 54
    , min_samples_split = 6
    , min_samples_leaf = 1
    , criterion = 'entropy'
    , class_weight = 'balanced'
    , random_state=RANDOM_STATE
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
rf_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

In [ ]:
# 소프트 보팅 결과
final_predictions = voting(rf_probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data09xx_rf.csv", index=False)

### ada

In [ ]:
base_estimator = DecisionTreeClassifier(
    max_depth=22,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features=0.9449544624225188,
    random_state=RANDOM_STATE
)

model_Dam = fit_all_train_data_function(
    'ada', train_data_dam
    , estimator=base_estimator
    , n_estimators=439
    , learning_rate=0.30985993372769294
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=13,
    min_samples_split=34,
    min_samples_leaf=5,
    max_features=0.7473309094470472,
    random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'ada', train_data_autoclave
    , estimator=base_estimator
    , n_estimators=570
    , learning_rate=0.2040105705276999
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=14,
    min_samples_split=33,
    min_samples_leaf=8,
    max_features=0.7113128413756866,
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'ada', train_data_fill1
    , estimator=base_estimator
    , n_estimators=913
    , learning_rate=0.055237331816147595
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=7,
    min_samples_split=13,
    min_samples_leaf=8,
    max_features=0.6266118401157937,
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'ada', train_data_fill2
    , estimator=base_estimator
    , n_estimators=293
    , learning_rate=0.620377973483163
    , random_state=RANDOM_STATE
)

base_estimator = DecisionTreeClassifier(
    max_depth=6,
    min_samples_split=28,
    min_samples_leaf=7,
    max_features=0.7331591188366589,
    random_state=RANDOM_STATE
)

model_All = fit_all_train_data_function(
    'ada', train_data
    , estimator=base_estimator
    , n_estimators=677
    , learning_rate=0.6713565955468803
    , random_state=RANDOM_STATE
)

# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
x_test_all = test_data.drop(["target", "Set ID"], axis=1)

# 예측 확률 리스트 (소프트 보팅용)
ada_probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    , model_All.predict_proba(x_test_all)[:, 1]
]

In [ ]:
# 소프트 보팅 결과
final_predictions = voting(ada_probs, method='soft', threshold=0.3)
print(sum(final_predictions))

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("./data/data09xx_ada.csv", index=False)

## Hard voting

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    
    # 각 샘플의 예측 결과를 문자열로 변환하여 리스트에 저장
    sample_predictions = [''.join(map(str, x)) for x in preds.T]
    
    # 각 예측 결과의 빈도수를 계산
    prediction_counts = Counter(sample_predictions)
    
    # 빈도수 출력
    for pred, count in prediction_counts.items():
        print(f"Prediction {pred}: {count} times")
    
    # 하드 보팅을 통해 최종 예측을 계산
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

In [ ]:
import pandas as pd

# 공통 경로
common_path = "./data/"

# 제출 파일 이름 리스트
file_names = [
    "data09xx_lgbm.csv"
    , "data09xx_xgb.csv"
    , "data09xx_cat.csv"
    , "data09xx_et.csv"
    , "data09xx_rf.csv"
    , "data09xx_ada.csv"   
    # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
]

# 경로를 추가하는 함수
def add_common_path(file_names, common_path):
    return [common_path + file_name for file_name in file_names]

# 경로가 추가된 파일 리스트
file_paths = add_common_path(file_names, common_path)

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장할 파일 이름
output_file_name = "change_name_submission.csv" # <----- 파일 이름을 변경하세요!!

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub["target"] = final_predictions_hard
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub.to_csv(output_file_name, index=False)

print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

In [ ]:
df_sub['target'].value_counts()

In [ ]:
df_sub.head(10)

우측 상단의 제출 버튼을 클릭해 결과를 확인하세요

.